In [1]:
import scipy

In [3]:
import anndata
import scanpy as sc
import pandas as pd
import scipy.io as sio

# import raw

In [4]:
raw = sc.read_h5ad('../00.raw/covid_portal_210320_with_raw.h5ad')

# data cleaning

In [5]:
raw.obs.columns

Index(['sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts',
       'total_counts_mt', 'pct_counts_mt', 'full_clustering',
       'initial_clustering', 'Resample', 'Collection_Day', 'Sex',
       'Age_interval', 'Swab_result', 'Status', 'Smoker',
       'Status_on_day_collection', 'Status_on_day_collection_summary',
       'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status',
       'Outcome', 'patient_id'],
      dtype='object')

In [6]:
raw.obs

,sample_id,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,full_clustering,initial_clustering,Resample,Collection_Day,...,Status,Smoker,Status_on_day_collection,Status_on_day_collection_summary,Days_from_onset,Site,time_after_LPS,Worst_Clinical_Status,Outcome,patient_id
covid_index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCTA-MH9179824,MH9179824,1160,1160,2172.0,79.0,3.637201,CD8.TE,CD8,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTAATC-MH9179824,MH9179824,1419,1419,3947.0,54.0,1.368128,CD4.IL22,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTGAGA-MH9179824,MH9179824,1986,1986,5886.0,143.0,2.429494,CD8.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGGAATCGC-MH9179824,MH9179824,1508,1508,4271.0,106.0,2.481854,CD4.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGTGTTGAA-MH9179824,MH9179824,1109,1109,2279.0,76.0,3.334796,CD8.EM,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BGCV15_TTTGTCAAGGCGATAC-1,BGCV15_CV0176,2401,2401,8633.0,490.0,5.675895,CD4.CM,CD4,Resample,D28,...,Covid,Non-smoker,ITU_O2,Severe,31,Cambridge,nan,Critical,unknown,CV0176
BGCV15_TTTGTCACAAGCCATT-1,BGCV15_CV0944,1973,1973,5315.0,103.0,1.937912,NK_16hi,NK_16hi,Initial,D0,...,Healthy,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944
BGCV15_TTTGTCACAGACACTT-1,BGCV15_CV0257,1209,1209,3345.0,230.0,6.875935,CD8.EM,CD8,Resample,D28,...,Covid,Non-smoker,Ward_O2,Moderate,13,Cambridge,nan,Moderate,Home,CV0257


In [7]:
proc = raw

In [8]:
# initial collection
proc = proc[proc.obs["Resample"] == "Initial"]

In [9]:
# covid/healthy at collection day
proc = proc[~proc.obs["Status_on_day_collection_summary"].isin(['Non_covid', 'LPS_90mins', "LPS_90mins", "LPS_10hours"])]

In [10]:
proc = proc[~proc.obs["initial_clustering"].isin(['Mono_prolif'])]
#proc = proc[proc.obs["Smoker"].isin(['Non-smoker'])]

In [11]:
proc = proc[proc.obs["Status_on_day_collection_summary"].isin(["Mild", "Severe"])]

In [12]:
pd.crosstab(proc.obs.initial_clustering, proc.obs.Status_on_day_collection_summary, margins=True, margins_name="Total")

Status_on_day_collection_summary,Mild,Severe,Total
initial_clustering,,,
B_cell,14739,8801,23540
CD4,32021,15037,47058
CD8,17856,10668,28524
CD14,25496,13416,38912
CD16,5215,1523,6738
DCs,1608,516,2124
HSC,453,781,1234
Lymph_prolif,1446,996,2442
MAIT,1597,360,1957


In [41]:
#proc = proc[~proc.obs["initial_clustering"].isin(['B_cell', 'CD4', 'CD8', "CD14", "CD16", "DCs", "NK_16hi",])]


# write out

In [13]:
proc.X

<204649x24929 sparse matrix of type '<class 'numpy.float32'>'
	with 290859700 stored elements in Compressed Sparse Row format>

In [14]:
#X = pd.DataFrame(data=scipy.sparse.csr_matrix.todense(proc.X))
#X.to_csv('../99.processed/counts.csv', index=False)
sio.mmwrite("../99.processed/counts_MildSevere.mtx", proc.X)

In [15]:
proc.obs

,sample_id,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,full_clustering,initial_clustering,Resample,Collection_Day,...,Status,Smoker,Status_on_day_collection,Status_on_day_collection_summary,Days_from_onset,Site,time_after_LPS,Worst_Clinical_Status,Outcome,patient_id
covid_index,,,,,,,,,,,,,,,,,,,,,
AAACCTGCACAAGCCC-MH9143325,MH9143325,864,864,1572.0,7.0,0.445293,NK_16hi,NK_16hi,Initial,D0,...,Covid,Non-smoker,ITU_NIV,Severe,7,Ncl,nan,Death,Death,MH9143325
AAACCTGCACAGGAGT-MH9143325,MH9143325,778,778,1327.0,15.0,1.130369,CD4.CM,CD4,Initial,D0,...,Covid,Non-smoker,ITU_NIV,Severe,7,Ncl,nan,Death,Death,MH9143325
AAACCTGCACCAGTTA-MH9143325,MH9143325,464,464,801.0,18.0,2.247191,CD14_mono,CD14,Initial,D0,...,Covid,Non-smoker,ITU_NIV,Severe,7,Ncl,nan,Death,Death,MH9143325
AAACCTGCACGGCCAT-MH9143325,MH9143325,554,554,906.0,8.0,0.883002,NK_16hi,NK_16hi,Initial,D0,...,Covid,Non-smoker,ITU_NIV,Severe,7,Ncl,nan,Death,Death,MH9143325
AAACCTGCACTTCGAA-MH9143325,MH9143325,828,828,1542.0,29.0,1.880674,CD4.Naive,CD4,Initial,D0,...,Covid,Non-smoker,ITU_NIV,Severe,7,Ncl,nan,Death,Death,MH9143325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BGCV13_TTTGCGCAGACCTTTG-1,BGCV13_CV0326,2845,2845,11372.0,76.0,0.668308,HSC_CD38pos,HSC,Initial,D0,...,Covid,Non-smoker,Ward_noO2,Mild,8,Cambridge,nan,Mild,Home,CV0326
BGCV13_TTTGCGCAGAGTCGGT-1,BGCV13_CV0326,1241,1241,3154.0,14.0,0.443881,CD4.CM,CD4,Initial,D0,...,Covid,Non-smoker,Ward_noO2,Mild,8,Cambridge,nan,Mild,Home,CV0326
BGCV13_TTTGCGCCACTGTGTA-1,BGCV13_CV0326,1192,1192,2500.0,18.0,0.720000,NK_16hi,NK_16hi,Initial,D0,...,Covid,Non-smoker,Ward_noO2,Mild,8,Cambridge,nan,Mild,Home,CV0326


In [16]:
proc.obs.to_csv('../99.processed/metadata_MildSevere.csv', index=True)

In [17]:
features = proc.var

In [55]:
features.to_csv('../99.processed/features.csv', index=True)